# Case Study - Recommender EDA

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

In [6]:
# Target output
target = pd.read_csv('data/sample_submission.csv')
target.head()

,user_id,joke_id,rating
0,49541,113,5
1,39499,37,5
2,36804,73,5
3,16470,33,5
4,22614,128,5


In [10]:
ratings_df = pd.read_table('data/ratings.dat')
ratings_df.head()

,user_id,joke_id,rating
0,34888,111,-9.71875
1,49651,119,4.75000
2,29027,137,2.62500
3,25235,30,-2.43750
4,50499,19,2.18750


In [15]:
jokes = pd.read_html('data/jokes.dat')

ImportError: html5lib not found, please install it